In [1]:
try:
  # for google colab users
    import google.colab # type: ignore
    from google.colab import output
    COLAB = True
    %pip install sae-lens transformer-lens
except:
  # for local setup
    COLAB = False
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# general imports
import os
import sys
import torch
from tqdm import tqdm
import plotly.express as px

torch.set_grad_enabled(False);

import copy
import pandas as pd
import itertools as ittl
import numpy as np
import math
from scipy import stats
from scipy.spatial import distance
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from datasets import Dataset
from datasets import load_dataset, concatenate_datasets
import transformers


In [2]:
# package import
from torch import Tensor
from transformer_lens import utils
from transformers import AutoTokenizer, AutoModelForCausalLM
from functools import partial
from jaxtyping import Int, Float


# device setup
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

Device: cuda
NVIDIA A40
NVIDIA A40


In [3]:
from transformer_lens import HookedTransformer
from sae_lens import SAE
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

In [4]:
import torch.distributed as dist
import torch.multiprocessing as mp

In [5]:
# get model
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

/home/jovyan/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [6]:
responses = []
def run_inference(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

    model.to(rank)

    if torch.distributed.get_rank() == 0:
        prompt = "a dog"
    elif torch.distributed.get_rank() == 1:
        prompt = "a cat"

    response = model.generate(prompt, max_new_tokens=10)
    responses.append(response)
    
def parallel_inference():
    world_size = 2
    mp.spawn(run_inference, args=(world_size,), nprocs=world_size, join=True)

In [7]:
parallel_inference()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/jovyan/conda-envs/steering/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/jovyan/conda-envs/steering/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'run_inference' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/jovyan/conda-envs/steering/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/jovyan/conda-envs/steering/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'run_inference' on <module '__main__' (built-in)>


ProcessExitedException: process 1 terminated with exit code 1